In [1]:
import os
from collections import Counter
import nltk
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from pymystem3 import Mystem
m = Mystem()
from gensim import utils

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/anna_mikhneva_/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
negative = []
for filename in os.listdir('neg_test'):
    f = os.path.join('neg_test', filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        with open(f, 'r') as file:
            for text in file:
                for word in nltk.word_tokenize(text):
                    if word.isalpha() and word[0].isupper() == False:
                        finword = morph.parse(word)[0]
                        negative.append(finword[0])
# print(negative)

In [19]:
positive = []
for filename in os.listdir('pos_test'):
    f = os.path.join('pos_test', filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        with open(f, 'r') as file:
            for text in file:
                for word in nltk.word_tokenize(text):
                    if word.isalpha()  and word[0].isupper() == False:
                        finword = morph.parse(word)[0]
                        positive.append(finword[0])
# print(positive)

In [20]:
freqlist = Counter()
for word in positive:
    freqlist[word] += 1
# print(freqlist)

In [21]:
freqlist_2 = Counter()
for word in negative:
    freqlist_2[word] += 1
# print(freqlist_2)

In [22]:
freq_pos = {}
freq_neg = {}
for key, value in freqlist.items():
    if key not in freqlist_2.keys() and value != 1: #убрала те, которые встретились по одному разу
        freq_pos[key] = value
freq_pos = sorted(freq_pos.items(), key=lambda item: item[1], reverse=True)
# print(dict(freq_pos))
for key, value in freqlist_2.items():
    if key not in freqlist.keys() and value != 1:
        freq_neg.update({key: value})
freq_neg = sorted(freq_neg.items(), key=lambda item: item[1], reverse=True)
# print(dict(freq_neg))

In [23]:
freq_lists = {}
freq_lists['pos'] = dict(freq_pos)
freq_lists['neg'] = dict(freq_neg)
print(freq_lists)

{'pos': {'wonderful': 32, 'priest': 32, 'lion': 15, 'strength': 12, 'masterpiece': 12, 'singing': 12, 'subtle': 11, 'vampires': 11, 'lesbian': 10, 'desires': 10, 'superb': 10, 'erotic': 9, 'outstanding': 9, 'vampirism': 9, 'details': 8, 'portrayed': 8, 'trilogy': 8, 'training': 8, 'families': 8, 'lust': 8, 'tragedy': 8, 'raid': 8, 'era': 8, 'boarding': 8, 'hockey': 8, 'freedom': 7, 'marriages': 7, 'reminds': 7, 'magnificent': 7, 'wonderfully': 7, 'cranes': 7, 'community': 7, 'circus': 7, 'moved': 6, 'finest': 6, 'marvelous': 6, 'dynamic': 6, 'husbands': 6, 'survive': 6, 'greatest': 6, 'technique': 6, 'genres': 5, 'taboo': 5, 'gender': 5, 'ordinary': 5, 'household': 5, 'ladies': 5, 'existence': 5, 'wives': 5, 'sacrifice': 5, 'responsibility': 5, 'ages': 5, 'wicked': 5, 'remains': 5, 'extraordinary': 5, 'cage': 5, 'tricks': 5, 'birth': 5, 'ensemble': 5, 'latter': 5, 'remarkable': 5, 'helping': 5, 'worthy': 5, 'animated': 5, 'skeletons': 5, 'movement': 5, 'backstage': 5, 'alcohol': 5, 'je

In [24]:
def tone_detect(freq_lists, text):
    counts = Counter()
    for tone, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[tone] += int(freq_list[word] > 0)
    return counts.most_common()

In [25]:
file = open('19_10.txt', 'r')
test = file.read()
tone_detect(freq_lists, test)

[('pos', 9), ('neg', 2)]

In [26]:
from sklearn.metrics import accuracy_score

In [31]:
results = []
check = []
for filename in os.listdir('check'):
    f = os.path.join('check', filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        with open(f, 'r') as file:
            for text in file:
                predicted_tone = tone_detect(freq_lists, text)[0][0]
                results.append(predicted_tone)
                check.append(filename[:3])
print(results)
print(check)
print("Accuracy: %.4f" % accuracy_score(results, check))

['neg', 'neg', 'neg', 'neg', 'pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neg']
['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
Accuracy: 0.8667


### Способ №1 (выполнен)
Убрать все имена собственные. Пропали также и первые слова в предложениях, но это можно решить токенизацией по предложением и проверкой на первые слова в предложении. (это я не успела сделать), но функция лучше работает без имен собственных, так как до этого в списке, например, было 37 Veronica и 12 Veronika из какого-то конкретного отзыва, а также ушли фамилии режиссеров.

### Способ №2 
Попробовать разбить на биграммы или триграммы. И работать уже с частотным списком n-grams. Возможно, получится отловить конструкции с very или конструкции с not.